In [2]:
from init import *

## load shader 'numbers'

In [14]:
def get_shader_values(qset=None):
    fields = (
        "shader_id", "num_views", "num_likes", 
        "num_views_per_day", "num_likes_per_day", 
        "num_passes", "num_characters",
        "num_lines_blank", "num_lines_code", "num_lines_comment",
        "num_chars_code", "num_chars_comment",
    )
    qset = qset or ShadertoyShader.objects.filter(num_views__gte=0)
    rows = qset.values_list(*fields)
    
    fields = [f.replace("num_lines", "nl").replace("num_chars", "nc").replace("num_", "") for f in fields]
    
    array = np.asarray(rows)
    df = pd.DataFrame(array, columns=fields)
    df.index = df[fields[0]]
    del df[fields[0]]
    for f in fields[1:]:
        df[f] = pd.to_numeric(df[f])
    return df
values = get_shader_values()
print("num shaders: %s" % len(values))
values

num shaders: 9441


,views,likes,views_per_day,likes_per_day,passes,characters,nl_blank,nl_code,nl_comment,nc_code,nc_comment
shader_id,,,,,,,,,,,
ls3fD7,437,7,1.908297,0.030568,6,53124,257,1118,170,33501,7053
ltKXRy,436,12,0.624642,0.017192,2,1762,18,44,6,1252,176
4ttXDX,436,4,0.595628,0.005464,3,15346,142,365,26,10273,1092
llK3zt,436,10,0.547051,0.012547,1,1970,24,36,11,1120,365
XdGfRR,436,24,2.116505,0.116505,2,4728,34,134,25,2058,1851
Msffzs,436,12,0.791289,0.021779,2,6133,8,116,56,1752,846
4stGWn,435,1,0.390836,0.000898,1,335,5,9,0,263,0
XdVXWd,435,11,0.483871,0.012236,1,549,6,11,3,308,80
XlcGzn,435,9,0.484410,0.010022,1,4872,19,122,33,2541,1088


## correlations

In [19]:
corr = values.corr()
corr.apply(lambda col: col.apply(lambda v: v if abs(v) > .2 else ""))

,views,likes,views_per_day,likes_per_day,passes,characters,nl_blank,nl_code,nl_comment,nc_code,nc_comment
views,1,0.792292,0.451858,,,,,,,,
likes,0.792292,1,0.405794,0.208246,,,0.205387,,,,
views_per_day,0.451858,0.405794,1,0.660172,,,,,,,
likes_per_day,,0.208246,0.660172,1,,,,,,,
passes,,,,,1,0.358311,0.343741,0.377964,,0.360941,
characters,,,,,0.358311,1,0.83161,0.925584,0.69846,0.964045,0.616731
nl_blank,,0.205387,,,0.343741,0.83161,1,0.839293,0.663646,0.750291,0.575261
nl_code,,,,,0.377964,0.925584,0.839293,1,0.540246,0.908043,0.440892
nl_comment,,,,,,0.69846,0.663646,0.540246,1,0.508288,0.955417
nc_code,,,,,0.360941,0.964045,0.750291,0.908043,0.508288,1,0.420262
